# [Exp-08]NaverMovieSentiment 

## 라이브러리 불러오기

In [1]:
import pandas as pd
import konlpy
import gensim
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

print(pd.__version__)
print(konlpy.__version__)
print(gensim.__version__)

1.4.3
0.6.0
4.2.0


### Tensorflow GPU 확인

In [2]:
# import tensorflow as tf
from tensorflow.python.client import device_lib

print(tf.config.list_physical_devices('GPU'))
device_lib.list_local_devices()

[]


2022-08-18 21:36:49.648800: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-08-18 21:36:49.648828: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: aiffel-GE75-Raider-10SF
2022-08-18 21:36:49.648833: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: aiffel-GE75-Raider-10SF
2022-08-18 21:36:49.648919: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.85.2
2022-08-18 21:36:49.648935: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.85.2
2022-08-18 21:36:49.648939: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.85.2
2022-08-18 21:36:49.650052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructi

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18080274224392936032
 xla_global_id: -1]

## 데이터 불러오기

[Naver sentiment movie corpus](https://github.com/e9t/nsmc)
* id : 네이버 아이디
* document : 실제 리뷰 내용
* label : 감성(0: 부정, 1: 긍정)

In [3]:
train_data = pd.read_table('data/ratings_train.txt')
test_data = pd.read_table('data/ratings_test.txt')

display(train_data.head())
display(test_data.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [4]:
print(train_data.shape)
print(len(train_data.label.unique()))
print(train_data.duplicated().sum())
train_data.info()

(150000, 3)
2
0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


* train 데이터는 15만개, 3개의 열을 가지고 있습니다.
* label의 uniuqe가 2개니 0,1 값만 가지고 있는게 맞습니다.
* document에서 null value를 가지고 있습니다.
* dtype은 document는 object이며(아마 string 같습니다.) 나머진 int 값입니다.

In [5]:
print(test_data.shape)
print(len(test_data.label.unique()))
print(test_data.duplicated().sum())
test_data.info()

(50000, 3)
2
0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


* test 데이터도 5만개, 3개의 열을 가지고 있습니다.
* 나머지 특징은 train과 같습니다.

## 데이터 로더 구성

* [Mecab 사용법](https://konlpy-ko.readthedocs.io/ko/v0.4.3/api/konlpy.tag/)

In [6]:
from konlpy.tag import Mecab
import numpy as np
from collections import Counter

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words = 10000):
    # 중복 값 제거
    train_data.drop_duplicates(subset=['document'], inplace=True)
    train_data = train_data.dropna(how = 'any') 
    test_data.drop_duplicates(subset=['document'], inplace=True)
    test_data = test_data.dropna(how = 'any') 
    
    # Mecab의 morphs가 string 단위로 읽어오기 때문에
    # 문장으로 받은 뒤 단어를 보면서 불용어인 경우에는 넣지 않습니다.
    X_train = []
    for sentence in train_data['document']:
        temp_X = tokenizer.morphs(sentence)
        temp_X = [word for word in temp_X if not word in stopwords]
        X_train.append(temp_X)

    X_test = []
    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence)
        temp_X = [word for word in temp_X if not word in stopwords]
        X_test.append(temp_X)
    
    # 문장들을 연결한뒤 리스트로 바꿔줍니다.
    words = np.concatenate(X_train).tolist()
    # 리스트의 요소와 개수를 
    counter = Counter(words)
    # 가장 많이 나온 것들 중들에서 (사용할 단어 - 4)개 만큼 가져온다.
    counter = counter.most_common(num_words - 4)
    # 단어 리스트를 만듭니다.
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]
    # 단어:index로 이루어진 딕셔너리를 만듭니다.
    word_to_index = {word:index for index, word in enumerate(vocab)}
    
    # wordlist를 indexlist로 바꾸는 함수입니다.(단어사전에 없으면 2(<UNK> 인덱스)를 넣습니다.)
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]
    
    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))
    
    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index


In [ ]:
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

In [7]:
index_to_word = {index:word for word, index in word_to_index.items()}

In [8]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

## 모델 구성을 위한 데이터 분석 및 가공

In [14]:
print(type(X_train), type(X_test))
print(len(X_train), len(X_test))
print(X_train[0])
print(X_test[0])
print(type(y_train), type(y_test))
print(len(y_train), len(y_test))
print(y_train[0])
print(y_test[0])

<class 'list'> <class 'list'>
146182 49157
[32, 74, 919, 4, 4, 39, 228, 20, 33, 748]
[832, 130]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
146182 49157
0
1


### 데이터셋 내 문장 길이 분포

In [1]:
total_train_data = list(X_train) + list(X_test)
num_tokens = [len(tokens) for tokens in total_train_data]
num_tokens = np.array(num_tokens)
num_tokens_count = np.bincount(num_tokens)

print('num_tokens :', num_tokens)
print('문장길이 평균 :', np.mean(num_tokens))
print('문장길이 중간값 :', np.median(num_tokens))
print('문장길이 최대 :', np.max(num_tokens))
print('문장길이 표준편차: ', np.std(num_tokens))
print('문장길이 최빈값: ', num_tokens_count.argmax())


plt.figure(figsize = (10,5))
sns.histplot(num_tokens)
plt.show()

NameError: name 'X_train' is not defined